Cargamos las librerias

In [1]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import eegraph
import stellargraph as sg
from stellargraph import StellarGraph
import pickle

In [2]:
chs1=['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'Fc1', 'Fc5', 
      'Cp1', 'Cp5', 'F9', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 
      'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10']

In [3]:
chs2=['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'Fc1', 'Fc5', 
        'Cp1', 'Cp5', 'F9', 'Fz', 'Cz', 'Pz', 'F4', 'C4', 'P4', 
        'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10', 'Fp2']

In [4]:
chs3=['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T5', 'Fc1', 'Fc5',
         'Cp1', 'Cp5', 'F9', 'Fz', 'Pz', 'F4', 'C4', 'P4', 
         'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10', 
         'Fp2','T3', 'Cz']

In [5]:
chs_ex=['EKG EKG', 'SPO2', 'HR', '1', '2', 'MK','PLET', '61', '62', '63', '64', '3', '23', '33', '31', 'A', 'B', 
               'C', 'D', 'EEG P9','EEG P10','EKG', 'P9','P10']

In [6]:
path_master='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/'
path_save='/media/ceiec08/ESD-USB/Grafos_Export/'

## Funciones

Funcion para extraer las medidas de conectividad en las que no es necesario especificar las bandas de frecuencia

In [7]:
path_PN00='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage1/PN00/'
path_PN01='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage1/PN01/'
path_PN03='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN03/'
path_PN05='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN05/'
path_PN06='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN06/'
path_PN07='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN07/'
path_PN09='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN09/'
path_PN10='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN10/'
path_PN11='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN11/'
path_PN12='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN12/'
path_PN13='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage2/PN13/'
path_PN14='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage3/PN14/'
path_PN16='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage3/PN16/'
path_PN17='/media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage3/PN17/'


In [8]:
def read_eeg_no_bands(window, connectivity_measure, threshold, 
                      path_load,path_save, file, age, sex, channels,elm):
    list_graphs=[]
    G = eegraph.Graph()
    G.load_data(path = (path_load+file), exclude = chs_ex,
            electrode_montage_path = (path_master+elm))
    loop_graphs, loop_cm = G.modelate(window_size = window, 
                               connectivity = connectivity_measure, 
                               threshold=threshold)  
    for i in loop_graphs:
        gri=loop_graphs[i]
        age_list=[age]*len(gri)
        sex_list=[sex]*len(gri)
        feat=pd.DataFrame(gri.degree(), index=channels, 
                columns=['node','degree']).drop('node', axis=1)
        feat['age']=age_list
        feat['female']=sex_list
        df_gri=nx.convert_matrix.to_pandas_edgelist(gri)
        df_gri['source']=np.where(df_gri['source']=='FP2','Fp2',df_gri['source'])
        df_gri['target']=np.where(df_gri['target']=='FP2','Fp2',df_gri['target'])
        df_gri['source']=np.where(df_gri['source']=='CZ','Cz',df_gri['source'])
        df_gri['target']=np.where(df_gri['target']=='CZ','Cz',df_gri['target'])
        st_graph=StellarGraph(feat, df_gri)
        list_graphs.append(st_graph)               
    return list_graphs

In [9]:
def read_folder(path, age, ssex, em):
    if em==1:
        channels=chs1
        elm="electrodemontage.set.ced"
    elif em==2:
        channels=chs2
        elm="electrodemontage2.set.ced"
    elif em==3:
        channels=chs3
        elm="electrodemontage3.set.ced"
        
    if ssex.lower()=='female':
        sex=1
    else:
        sex=0
    files=sorted(os.listdir(path))
    total_graphs=[]
    for f in files:
        list_graphs=read_eeg_no_bands(4,'pearson_correlation', 
                    0.7, path, path_save,f,age,sex,channels,elm)
        total_graphs=total_graphs+list_graphs
    return total_graphs

In [10]:
def save_sg_list(sglist,path_save, filename):
    open_file=open(path_save+filename,'wb')
    pickle.dump(sglist, open_file)
    open_file.close()

In [ ]:
graphs_PN00=read_folder(path_PN00, 55, 'male',1)
save_sg_list(graphs_PN00, path_save, 'graphs_PN00_deg_sex_age.sg')
graphs_PN01=read_folder(path_PN01, 46, 'male',1)
save_sg_list(graphs_PN01, path_save, 'graphs_PN01_deg_sex_age.sg')
graphs_PN03=read_folder(path_PN03, 54, 'male',2)
save_sg_list(graphs_PN03, path_save, 'graphs_PN03_deg_sex_age.sg')
graphs_PN05=read_folder(path_PN05, 51, 'female',2)
save_sg_list(graphs_PN05, path_save, 'graphs_PN05_deg_sex_age.sg')
graphs_PN06=read_folder(path_PN06, 36, 'male',2)
save_sg_list(graphs_PN06, path_save, 'graphs_PN06_deg_sex_age.sg')
graphs_PN07=read_folder(path_PN07, 20, 'female',2)
save_sg_list(graphs_PN07, path_save, 'graphs_PN07_deg_sex_age.sg')
graphs_PN09=read_folder(path_PN09, 27, 'female',2)
save_sg_list(graphs_PN09, path_save, 'graphs_PN09_deg_sex_age.sg')
graphs_PN10=read_folder(path_PN10, 25, 'male',2)
save_sg_list(graphs_PN10, path_save, 'graphs_PN10_deg_sex_age.sg')
graphs_PN11=read_folder(path_PN11, 58, 'female',2)
save_sg_list(graphs_PN11, path_save, 'graphs_PN11_deg_sex_age.sg')
graphs_PN12=read_folder(path_PN12, 71, 'male',2)
save_sg_list(graphs_PN12, path_save, 'graphs_PN12_deg_sex_age.sg')
graphs_PN13=read_folder(path_PN13, 34, 'female',2)
save_sg_list(graphs_PN13, path_save, 'graphs_PN13_deg_sex_age.sg')
graphs_PN14=read_folder(path_PN14, 49, 'male',3)
save_sg_list(graphs_PN14, path_save, 'graphs_PN14_deg_sex_age.sg')
graphs_PN16=read_folder(path_PN16, 41, 'female',3)
save_sg_list(graphs_PN16, path_save, 'graphs_PN16_deg_sex_age.sg')
graphs_PN17=read_folder(path_PN17, 42, 'male',3)
save_sg_list(graphs_PN17, path_save, 'graphs_PN17_deg_sex_age.sg')

Extracting EDF parameters from /media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage1/PN00/PN00-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 29
Sample rate: 512.0 Hz.
Duration: 2624.998 seconds.
Channel Names: ['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'Fc1', 'Fc5', 'Cp1', 'Cp5', 'F9', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 2048.0), (2048.0, 4096.0), (4096.0, 6144.0), (6144.0, 8192.0), (8192.0, 10240.0), (10240.0, 12288.0), (12288.0, 14336.0), (14336.0, 16384.0), (16384.0, 18432.0), (18432.0, 20480.0), (20480.0, 22528.0), (22528.0, 24576.0), (24576.0, 26624.0), (26624.0, 28672.0), (28672.0, 30720.0), (30720.0, 32768.0), (32768.0, 34816.0), (34816.0, 36864.0), (36864.0, 38912.0), (38912.0, 40960.0), (40960.0, 43008.0), (43008.0, 45056.0), (45056.0, 47104.0), (47


Threshold: 0.7
Number of graphs created: 657
Extracting EDF parameters from /media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage1/PN00/PN00-2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 29
Sample rate: 512.0 Hz.
Duration: 2301.998 seconds.
Channel Names: ['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'Fc1', 'Fc5', 'Cp1', 'Cp5', 'F9', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 2048.0), (2048.0, 4096.0), (4096.0, 6144.0), (6144.0, 8192.0), (8192.0, 10240.0), (10240.0, 12288.0), (12288.0, 14336.0), (14336.0, 16384.0), (16384.0, 18432.0), (18432.0, 20480.0), (20480.0, 22528.0), (22528.0, 24576.0), (24576.0, 26624.0), (26624.0, 28672.0), (28672.0, 30720.0), (30720.0, 32768.0), (32768.0, 34816.0), (34816.0, 36864.0), (36864.0, 38912.0), (38912.0, 40960.0), (40960.0, 43008.0


Threshold: 0.7
Number of graphs created: 576
Extracting EDF parameters from /media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage1/PN00/PN00-3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 29
Sample rate: 512.0 Hz.
Duration: 2508.998 seconds.
Channel Names: ['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'Fc1', 'Fc5', 'Cp1', 'Cp5', 'F9', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 2048.0), (2048.0, 4096.0), (4096.0, 6144.0), (6144.0, 8192.0), (8192.0, 10240.0), (10240.0, 12288.0), (12288.0, 14336.0), (14336.0, 16384.0), (16384.0, 18432.0), (18432.0, 20480.0), (20480.0, 22528.0), (22528.0, 24576.0), (24576.0, 26624.0), (26624.0, 28672.0), (28672.0, 30720.0), (30720.0, 32768.0), (32768.0, 34816.0), (34816.0, 36864.0), (36864.0, 38912.0), (38912.0, 40960.0), (40960.0, 43008.0


Threshold: 0.7
Number of graphs created: 628
Extracting EDF parameters from /media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage1/PN00/PN00-4.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 29
Sample rate: 512.0 Hz.
Duration: 2101.998 seconds.
Channel Names: ['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'Fc1', 'Fc5', 'Cp1', 'Cp5', 'F9', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 2048.0), (2048.0, 4096.0), (4096.0, 6144.0), (6144.0, 8192.0), (8192.0, 10240.0), (10240.0, 12288.0), (12288.0, 14336.0), (14336.0, 16384.0), (16384.0, 18432.0), (18432.0, 20480.0), (20480.0, 22528.0), (22528.0, 24576.0), (24576.0, 26624.0), (26624.0, 28672.0), (28672.0, 30720.0), (30720.0, 32768.0), (32768.0, 34816.0), (34816.0, 36864.0), (36864.0, 38912.0), (38912.0, 40960.0), (40960.0, 43008.0


Threshold: 0.7
Number of graphs created: 526
Extracting EDF parameters from /media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage1/PN00/PN00-5.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 29
Sample rate: 512.0 Hz.
Duration: 2142.998 seconds.
Channel Names: ['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'Fc1', 'Fc5', 'Cp1', 'Cp5', 'F9', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 2048.0), (2048.0, 4096.0), (4096.0, 6144.0), (6144.0, 8192.0), (8192.0, 10240.0), (10240.0, 12288.0), (12288.0, 14336.0), (14336.0, 16384.0), (16384.0, 18432.0), (18432.0, 20480.0), (20480.0, 22528.0), (22528.0, 24576.0), (24576.0, 26624.0), (26624.0, 28672.0), (28672.0, 30720.0), (30720.0, 32768.0), (32768.0, 34816.0), (34816.0, 36864.0), (36864.0, 38912.0), (38912.0, 40960.0), (40960.0, 43008.0


Threshold: 0.7
Number of graphs created: 536
Extracting EDF parameters from /media/ceiec08/ESD-USB/siena-scalp-eeg-database-1.0.0/EEG/emontage1/PN01/PN01-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 29
Sample rate: 512.0 Hz.
Duration: 48556.998 seconds.
Channel Names: ['Fp1', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'T5', 'Fc1', 'Fc5', 'Cp1', 'Cp5', 'F9', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'T6', 'Fc2', 'Fc6', 'Cp2', 'Cp6', 'F10']
Model Data.
Pearson_correlation_Estimator()
Intervals:  [(0, 2048.0), (2048.0, 4096.0), (4096.0, 6144.0), (6144.0, 8192.0), (8192.0, 10240.0), (10240.0, 12288.0), (12288.0, 14336.0), (14336.0, 16384.0), (16384.0, 18432.0), (18432.0, 20480.0), (20480.0, 22528.0), (22528.0, 24576.0), (24576.0, 26624.0), (26624.0, 28672.0), (28672.0, 30720.0), (30720.0, 32768.0), (32768.0, 34816.0), (34816.0, 36864.0), (36864.0, 38912.0), (38912.0, 40960.0), (40960.0, 43008.

In [ ]:
graphs=graphs_PN00+graphs_PN01+graphs_PN03+graphs_PN05+graphs_PN06+graphs_PN07+graphs_PN09+graphs_PN10+graphs_PN11+graphs_PN12+graphs_PN13+graphs_PN14+graphs_PN16+graphs_PN17

In [ ]:
save_sg_list(graphs, path_save, 'graphs_pearson_deg_sex_age_all.sg')